In [2]:
import numpy as np
from numba import jit, njit
from scipy.sparse import bsr_array

In [3]:
def get_nq_and_eps(Nq_max, A, lamb, n1, n2):
    # Determine the parity of Nq - lamb
    Nqpar = Nq_max % 2
    Nq_minus_lamb_par = (Nqpar - lamb) % 2
    eps = Nq_minus_lamb_par % 2
    # Determine Nq
    Nq = 2 * n1 + 2 * n2 + 2 * A + lamb + eps

    return Nq, eps


def get_ells(A, B, lamb, eps):    
    l1 = A + B + eps
    l2 = A - B + lamb

    return l1, l2

# Based on the stupid order things are saved in FORTRAN (yes, makes no sense)
# From: N' (B'), n1', M' (A'), n1, n2, N (B), M (A), L (lamb)
# To: n1, l1, n2, l2, n1', l1', l2', lamb  (  n2' = (2*n1 + l1 + 2*n2 + l2 - 2*n1' - l1' - l2')/2  )
# Or: Ne1, l1, Ne2, l2, Ne1', l1', l2', lamb  (  Ne2' = Ne1 + Ne2 - Ne1'  )
def recast_indices(mode, Nq_max, Bprime, n1prime, Aprime, n1, n2, B, A, lamb):
    Nq, eps = get_nq_and_eps(Nq_max, A, lamb, n1, n2)
    l1, l2 = get_ells(A, B, lamb, eps)
    l1prime, l2prime = get_ells(Aprime, Bprime, lamb, eps)

    if mode == 'n':
        return [n1, l1, n2, l2, n1prime, l1prime, l2prime, lamb]
    
    elif mode == 'Ne':
        Ne1 = 2 * n1 + l1
        Ne2 = 2 * n2 + l2
        Ne1prime = 2 * n1prime + l1prime

        return [Ne1, l1, Ne2, l2, Ne1prime, l1prime, l2prime, lamb]



In [21]:
# Prepare the field
lamb_max = 10
lamb_min = 0
#nqdif = 2
Nq_max_even = 10
Nq_max_odd = 11
Nq_max = max(Nq_max_even, Nq_max_odd)

sz = (Nq_max - lamb_min)//2 + 1
brackets_f = np.zeros((lamb_max + 1, sz, sz, sz, sz, lamb_max + 1, sz, lamb_max - lamb_min + 1))

# Open the file and load the Fortran computed values
values_and_indices_f_even = np.loadtxt('OSBRACKETS/out_even.dat')
values_and_indices_f_odd = np.loadtxt('OSBRACKETS/out_odd.dat')
values_f = np.hstack((values_and_indices_f_even[:, -1], values_and_indices_f_odd[:, -1]))
indices_f = np.vstack((values_and_indices_f_even[:, :-1].astype(int), values_and_indices_f_odd[:, :-1].astype(int)))

# Index the matrix
# Based on the stupid order things are saved in FORTRAN (yes, makes no sense)
# N' (B'), n1', M' (A'), n1, n2, N (B), M (A), L (lamb)
brackets_f[tuple(indices_f.T)] = values_f

# --------------------------------------------
# Brackets with the quantum numbers we want:
# n1, l1, n2, l2, n1', l1', l2', lamb   (  n2' = (2*n1 + l1 + 2*n2 + l2 - 2*n1' - l1' - l2')/2  )
l_min = 0
l_max = Nq_max #lamb_max (there is an interesting reason why this is not lamb_max)
n_max = (Nq_max - lamb_min)//2
brackets = np.zeros((n_max + 1, l_max - l_min + 1, n_max + 1, l_max - l_min + 1, n_max + 1, 
                                l_max - l_min + 1, l_max - l_min + 1, lamb_max - lamb_min + 1))
# Recast the indices (incredibly works)
indices = np.vstack(recast_indices('n', Nq_max, *[indices_f[:, i] for i in range(8)])).T
# Index the matrix
brackets[tuple(indices.T)] = values_f

# Some tests
# print(indices_f[12])
# print(recast_indices('n', Nq_max, *indices_f[12]))
# print(indices[12])
# print(values_f[12], brackets[tuple(indices[12].T)])

# --------------------------------------------
# Or:
# Ne1, l1, Ne2, l2, Ne1', l1', l2', lamb  (  Ne2' = Ne1 + Ne2 - Ne1'  )
Ne_max = Nq_max
brackets_Ne = np.zeros((Ne_max + 1, l_max - l_min + 1, Ne_max + 1, l_max - l_min + 1, Ne_max + 1,
                                l_max - l_min + 1, l_max - l_min + 1, lamb_max - lamb_min + 1))
# Recast the indices
indices_Ne = np.vstack(recast_indices('Ne', Nq_max, *[indices_f[:, i] for i in range(8)])).T
# Index the matrix
brackets_Ne[tuple(indices_Ne.T)] = values_f

# Some tests
# print(indices_Ne[12])
# print(indices[12])
# print(brackets_Ne[tuple(indices_Ne[12].T)], brackets[tuple(indices[12].T)])

# These arrays are sparse as fuck, cast them to sparse matrices
#brackets_sp = bsr_array(brackets)
# print(np.count_nonzero(brackets_Ne)/brackets_Ne.size)
# print(np.count_nonzero(brackets)/brackets.size)


for i in range(0, len(values_f)):
    break
    idx = indices_f[i]

    n1 = idx[3]
    n2 = idx[4]
    lamb = idx[7]
    A, B = idx[6], idx[5]
    Nq, eps = get_nq_and_eps(Nq_max, A, lamb, n1, n2)
    l1, l2 = get_ells(A, B, lamb, eps)

    print(l1, l2, lamb)

    # if i % 1000 == 0:
    #     print(i)
        
    #print(l1, l2, l1 + l2)

    # Nq = 2 * n1 + l1 + 2 * n2 + l2
    # if (Nq_max - Nq) % 2 != 0:
    #     print("SDADd")

    # if Nq == 5:
    #     print(Nq_max - Nq, Nq, eps)
    #     print(lamb, l1, l2, n1, n2)
    #     print('---')
        # print(idx, values_f[i])

    # print(idx, values[i])
    # print(idx, brackets_f[tuple(idx.T)], "Idx")


In [22]:
# Save the numpy arrays
np.save('./saved_values/brackets.npy', brackets)
np.save('./saved_values/brackets_Ne.npy', brackets_Ne)

In [5]:
import ctypes

# Load the shared library
lib = ctypes.cdll.LoadLibrary('./wigner.so')
# Print the lib attributes and functions


# Declare the function prototype
# NineJSymbol( double J1, double J2, double J3, double J4, double J5, double J6, double J7, double J8, double J9)
lib.NineJSymbol.argtypes = [ctypes.c_double, ctypes.c_double, ctypes.c_double, ctypes.c_double, ctypes.c_double, ctypes.c_double, ctypes.c_double, ctypes.c_double, ctypes.c_double]
lib.NineJSymbol.restype = ctypes.c_double

# Call the function
print(lib.NineJSymbol(1, 1, 1, 1, 1, 1, 1, 1, 1))

-6.938893903907228e-18


In [1]:
# Will not use this for now as we already have it working in FORTRAN, but 
# can be useful for the future

import numpy as np
import OSBRACKETS.allos as allos
co = - 1 / np.sqrt(2)
si = - co

In [4]:
#print(allosbrac.__dict__)
lamb_max = 20
lamb_min = 0
Nq_max = 20
sz = (Nq_max - lamb_min) // 2 + 1
brackets_f = np.zeros((lamb_max + 1, sz, sz, sz, sz, lamb_max + 1, sz, lamb_max - lamb_min + 1))

print(allos.allosbrac.__doc__)

allos.allosbrac(Nq_max, lamb_min, lamb_max, co, si, brackets_f)

brac = allosbrac(nqmax,lmin,lmax,co,si,brac)

Wrapper for ``allosbrac``.

Parameters
----------
nqmax : input int
lmin : input int
lmax : input int
co : input float
si : input float
brac : input rank-8 array('d') with bounds (1 + lmax,1 + (-lmin + nqmax) / 2,1 + (-lmin + nqmax) / 2,1 + (-lmin + nqmax) / 2,1 + (-lmin + nqmax) / 2,1 + lmax,1 + (-lmin + nqmax) / 2,1 + lmax - lmin)

Returns
-------
brac : rank-8 array('d') with bounds (1 + lmax,1 + (-lmin + nqmax) / 2,1 + (-lmin + nqmax) / 2,1 + (-lmin + nqmax) / 2,1 + (-lmin + nqmax) / 2,1 + lmax,1 + (-lmin + nqmax) / 2,1 + lmax - lmin)

